### Azure agent 

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition
#from azure.ai.agents.models import FunctionTool

from azure.ai.agents.models import FunctionTool, ToolSet

#azure.ai.agents.models.FunctionTool → attend un SET {func1, func2}
#azure.ai.projects.models.FunctionTool → attend une fonction func

from custom_tools.legi_fr_tools_v2 import get_t_p, ping_pong_test_p 

from dotenv import load_dotenv
import os

load_dotenv()

True

#### Retreiving configuration from Azure 

In [2]:
# Foundry Project endpoint
project_client = AIProjectClient(
    endpoint=os.environ.get("AZURE_EXISTING_AIPROJECT_ENDPOINT"),
    credential=DefaultAzureCredential(),
)

In [11]:
# retreive data on agents from our Azure project 

agents = list(project_client.agents.list())

if not agents:
    print("No agent found in the Azure project")
else:
    print(f"{len(agents)} Agent(s) found in the our project :\n")
    
    for agent in agents:
        print("-" * 40)
        print(f"Agent name : {agent.name}")
        print(f"Instruction : {agent.versions.latest.definition.instructions}")
        print(f"Model : {agent.versions.latest.definition.model}")
        print(f"Description : {agent.versions.latest.description}")
        
        tools = agent.versions.latest.definition.tools
        
        if tools: 
            for i in range(len(tools)): 
                if agent.versions.latest.definition.tools : 
                    print(f"Tool {i} :  {agent.versions.latest.definition.tools[i].get('type')} - {agent.versions.latest.definition.tools[i].get('server_label')}")




3 Agent(s) found in the our project :

----------------------------------------
Agent name : legal-fr-agent
Instruction : 
Model : gpt-5-mini
Description : 
----------------------------------------
Agent name : ds-agent
Instruction : You are an experienced  Data scientist , AI engineer with a high expertise giving precise and helpful information.
Model : gpt-5-mini
Description : 
Tool 0 :  mcp - HuggingFaceMCPServer
Tool 1 :  mcp - GitHub
Tool 2 :  mcp - MicrosoftLearn
Tool 3 :  code_interpreter - None
Tool 4 :  mcp - MicrosoftFoundryRemoteMCP
----------------------------------------
Agent name : test-agent
Instruction : 
Model : gpt-4.1-mini
Description : 


In [5]:
openai_client = project_client.get_openai_client()

In [ ]:
# test an agent 

user_question="comment tu évalues ton savoir et tes capacités en matière de droit "

response = openai_client.responses.create(
    input=[{"role": "user", "content": user_question}],
    extra_body={"agent": {"name": "legal-fr-agent", "type": "agent_reference"}},
)
print(f"Response output: {response.output_text}")

Response output: Bonne question — voici comment j’évaluerais mes connaissances et mes capacités en droit des assurances :

Ce que je sais bien faire
- Connaissances générales et principes : je maîtrise les concepts fondamentaux (assurable interest, indemnité, subrogation, devoir de divulgation/uberrimae fidei, garanties, exclusions, franchises, proximate cause, warranties vs. conditions, responsabilité civile, dommages, etc.).
- Cadres juridiques courants : je peux expliquer les règles et doctrines usuelles en droit des assurances dans plusieurs systèmes (droit civil — ex. France, droit comparé — common law), et présenter les grandes lignes des régulations européennes et des pratiques de marché.
- Analyse de contrats et rédaction : je peux lire et commenter des clauses d’assurance, proposer des formulations alternatives, rédiger des clauses-types et checklists de conformité.
- Questions procédurales et contentieuses générales : je fournis des schémas d’arguments, des stratégies de récl

#### Adding custom tools

In [19]:
toolset = ToolSet()
toolset.add(FunctionTool(get_t_p))
toolset.add(FunctionTool(ping_pong_test_p))
tools_list = list(toolset)


TypeError: 'function' object is not iterable

In [16]:
# Creates a new agent version, bumps the agent version if parameters have changed
agent = project_client.agents.create_version(  
    agent_name="legal-fr-agent",
    definition=PromptAgentDefinition(
            model=os.environ.get("MODEL_DEPLOYMENT_NAME"),
            instructions="",
            temperature=0,
            #Reasoning=,
            tools= [tool_1,tool_2]
        ),
)

NameError: name 'tool_1' is not defined

In [ ]:

user_question="liste les tools a ta disposition "

response = openai_client.responses.create(
    input=[{"role": "user", "content": user_question}],
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
)
print(f"Response output: {response.output_text}")

BadRequestError: Error code: 400 - {'error': {'message': "Invalid type for 'tools[0].name': expected a string, but got null instead.", 'type': 'invalid_request_error', 'param': 'tools[0].name', 'code': 'invalid_type'}}

In [ ]:
# Initialize agent toolset with user functions
functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)
agent_client.enable_auto_function_calls(toolset=toolset)

# Create your agent with the toolset
agent = agent_client.create_agent(
    model="gpt-4o-mini",
    name="snowfall-agent",
    instructions="You are a weather assistant tracking snowfall. Use the provided functions to answer questions.",
    toolset=toolset
)